<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Papers/blob/main/FFSSD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import torch
import torchvision
from IPython.display import clear_output

!pip install torch_intermediate_layer_getter
from torch_intermediate_layer_getter import IntermediateLayerGetter

clear_output()
tf.__version__

'2.12.0'

#tf

In [5]:
# # vgg16 = tf.keras.applications.vgg16.VGG16(weights='imagenet')
# # vgg16.summary()
# # for i in range(len(vgg16.layers)):
# #   vgg16.layers[i].trainable=False
# #   print(vgg16.layers[i].name)
# def ftb(current_layer, former_layer=None):
#   x = tf.keras.layers.Conv2D(filters=256, kernel_size=3, strides=1, activation='relu', padding='same')(current_layer)
#   x = tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding='same')(x)

#   if former_layer is None:
#     return x
  
#   elif x.shape == former_layer.shape:
#     return tf.keras.layers.Add()([x, former_layer])

#   else:
#     d = tf.keras.layers.Conv2DTranspose(filters=256, kernel_size=4, strides=2, padding='same')(former_layer)
#     return tf.keras.layers.Add()([x, d])


# def rfem(x):
#   channel = x.shape[-1]
#   #branch1
  
#   x1 = tf.keras.layers.MaxPool2D(pool_size=2, padding='same', strides=1)(x)
#   x1 = tf.keras.layers.Conv2D(filters=channel//4, kernel_size=(1, 1), padding='same', activation='relu')(x1)
#   x1 = tf.keras.layers.BatchNormalization()(x1)

#   #branch2
#   x2 = tf.keras.layers.Conv2D(filters=channel//4, kernel_size=(1, 1), padding='same', activation='relu')(x)
#   x2 = tf.keras.layers.BatchNormalization()(x2)
#   x2 = tf.keras.layers.Conv2D(filters=channel//4, kernel_size=(1, 3), padding='same', activation='relu')(x2)
#   x2 = tf.keras.layers.BatchNormalization()(x2)

#   #branch3
#   x3 = tf.keras.layers.Conv2D(filters=channel//4, kernel_size=(1, 1), padding='same', activation='relu')(x)
#   x3 = tf.keras.layers.BatchNormalization()(x3)
#   x3 = tf.keras.layers.Conv2D(filters=channel//4, kernel_size=(3, 1), padding='same', activation='relu')(x3)
#   x3 = tf.keras.layers.BatchNormalization()(x3)

#   #branch4
#   x4 = tf.keras.layers.Conv2D(filters=channel//4, kernel_size=(1, 1), padding='same', activation='relu')(x)
#   x4 = tf.keras.layers.BatchNormalization()(x4)
#   x4 = tf.keras.layers.Conv2D(filters=channel//3, kernel_size=(3, 1), padding='same', activation='relu')(x4)
#   x4 = tf.keras.layers.BatchNormalization()(x4)
#   x4 = tf.keras.layers.Conv2D(filters=channel//3, kernel_size=(1, 3), padding='same', activation='relu')(x4)
#   x4 = tf.keras.layers.BatchNormalization()(x4)
#   x4 = tf.keras.layers.Conv2D(filters=channel//4, kernel_size=(3, 1), padding='same', activation='relu')(x4)
#   x4 = tf.keras.layers.BatchNormalization()(x4)
#   x4 = tf.keras.layers.Conv2D(filters=channel//4, kernel_size=(1, 3), padding='same', activation='relu')(x4)
#   x4 = tf.keras.layers.BatchNormalization()(x4)

#   combined_x = tf.keras.layers.Concatenate(axis=-1)([x1, x2, x3, x4])

#   return tf.keras.layers.Add()([combined_x, x/0.5])

In [8]:
# model = tf.keras.models.Model(inputs=input_x, outputs=[X])
# vgg16_short = tf.keras.models.Model(vgg16.input, vgg16.layers[-5].output)

# model_1 = tf.keras.Sequential([
#     vgg16_short, 
#     tf.keras.layers.Conv2D(filters=1024, kernel_size=(3, 3), padding='same', activation='relu', name='block6_conv1'),
#     tf.keras.layers.MaxPool2D(pool_size=2, name='block6_pool'),
#     tf.keras.layers.Conv2D(filters=512, kernel_size=(3, 3), strides=2, padding='same', activation='relu', name='block7_conv1'),
#     tf.keras.layers.MaxPool2D(pool_size=2, name='block7_pool'),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(units=4096, activation='relu', name='dense_layer'),
#     tf.keras.layers.Dense(units=7, activation='softmax', name='final_layer')
# ])
# model_1.summary()

# model_2 = tf.keras.models.Model(inputs=vgg16.input, outputs=[vgg16_short.get_layer('block4_conv2').output,
#                                                             vgg16_short.get_layer('block4_conv3').output,
#                                                             vgg16_short.get_layer('block5_conv3').output,
#                                                             model_1.get_layer('block6_conv1').output,
#                                                             model_1.get_layer('block7_conv1').output])

#Pytorch

In [11]:
class Encoder(torch.nn.Module):
  def __init__(self):
    super().__init__()
    model = torchvision.models.vgg16(weights='IMAGENET1K_V1')
    model.features = torch.nn.Sequential(*([model.features[i] for i in range(30)] + [model.features[i] for i in range(23, 30)]))
    return_layers = {'20': 'out_conv4_2', '22': 'out_conv4_3', '29': 'out_conv5_3','34': 'out_conv7_2', '36': 'out_conv6_2'}
    self.backbone = IntermediateLayerGetter(model.features, return_layers=return_layers)
    device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
  def forward(self, images):
    return self.backbone(images)

In [52]:
x = torch.rand(5,3,320,320)

encoder = Encoder()
y = encoder(x)

y[0].keys(), y[1].shape

(odict_keys(['out_conv4_2', 'out_conv4_3', 'out_conv7_2', 'out_conv5_3', 'out_conv6_2']),
 torch.Size([5, 512, 10, 10]))

In [31]:
def ftb(current_layer, former_layer=None):
  x = torch.nn.Conv2d(current_layer.shape[1], 256, kernel_size=3, stride=1, padding=1)(current_layer)
  x = torch.nn.ReLU(inplace=True)(x)
  x = torch.nn.Conv2d(256, 512, kernel_size=3, padding=1)(x)
  
  if former_layer is None:
    return x
  elif x.shape == former_layer.shape:
    return torch.add(x, former_layer)

  else:
    d = torch.nn.ConvTranspose2d(former_layer.shape[1], 512, kernel_size=4, stride=2, padding=1)(former_layer)
    return torch.add(x, d)

In [32]:
def rfem(x):
  channel = x.shape[1]
  #branch1
  
  x1 = torch.nn.Conv2d(in_channels=channel, out_channels=channel//4, kernel_size=(1, 1), padding='same')(x)
  x1 = torch.nn.ReLU()(x1)
  x1 = torch.nn.BatchNorm2d(num_features=channel//4)(x1)

  #branch2
  x2 = torch.nn.Conv2d(in_channels=channel, out_channels=channel//4, kernel_size=(1, 1), padding='same')(x)
  x2 = torch.nn.ReLU()(x2)
  x2 = torch.nn.BatchNorm2d(num_features=channel//4)(x2)
  x2 = torch.nn.Conv2d(in_channels=channel//4, out_channels=channel//4, kernel_size=(1, 3), padding='same')(x2)
  x2 = torch.nn.ReLU()(x2)
  x2 = torch.nn.BatchNorm2d(num_features=channel//4)(x2)

  #branch3
  x3 = torch.nn.Conv2d(in_channels=channel, out_channels=channel//4, kernel_size=(1, 1), padding='same')(x)
  x2 = torch.nn.ReLU()(x2)
  x3 = torch.nn.BatchNorm2d(num_features=channel//4)(x3)
  x3 = torch.nn.Conv2d(in_channels=channel//4, out_channels=channel//4, kernel_size=(3, 1), padding='same')(x3)
  x3 = torch.nn.ReLU()(x3)
  x3 = torch.nn.BatchNorm2d(num_features=channel//4)(x3)

  #branch4
  x4 = torch.nn.Conv2d(in_channels=channel, out_channels=channel//4, kernel_size=(1, 1), padding='same')(x)
  x4 = torch.nn.ReLU()(x4)
  x4 = torch.nn.BatchNorm2d(num_features=channel//4)(x4)
  x4 = torch.nn.Conv2d(in_channels=channel//4, out_channels=channel//3, kernel_size=(3, 1), padding='same')(x4)
  x4 = torch.nn.ReLU()(x4)
  x4 = torch.nn.BatchNorm2d(num_features=channel//3)(x4)
  x4 = torch.nn.Conv2d(in_channels=channel//3, out_channels=channel//3, kernel_size=(1, 3), padding='same')(x4)
  x4 = torch.nn.ReLU()(x4)
  x4 = torch.nn.BatchNorm2d(num_features=channel//3)(x4)
  x4 = torch.nn.Conv2d(in_channels=channel//3, out_channels=channel//4, kernel_size=(3, 1), padding='same')(x4)
  x4 = torch.nn.ReLU()(x4)
  x4 = torch.nn.BatchNorm2d(num_features=channel//4)(x4)
  x4 = torch.nn.Conv2d(in_channels=channel//4, out_channels=channel//4, kernel_size=(1, 3), padding='same')(x4)
  x4 = torch.nn.ReLU()(x4)
  x4 = torch.nn.BatchNorm2d(num_features=channel//4)(x4)

  combined_x = torch.concat((x1, x2, x3, x4), dim=1)

  return torch.add(combined_x, x/0.5)

In [50]:
def training(img):
  backbone = Encoder()
  feature_maps = backbone(img)

  conv4_2 = feature_maps[0]['out_conv4_2']
  conv4_2=torch.nn.Conv2d(in_channels=512, out_channels=512, kernel_size=2, stride=2)(conv4_2)
  conv4_2=torch.nn.ReLU()(conv4_2)
  conv4_3 = feature_maps[0]['out_conv4_3']
  conv4_3=torch.nn.Conv2d(in_channels=512, out_channels=512, kernel_size=2, stride=2)(conv4_3)
  conv4_3=torch.nn.ReLU()(conv4_3)
  conv5_3 = feature_maps[0]['out_conv5_3'][0]
  conv6_2 = feature_maps[0]['out_conv6_2'][0]
  conv7_2 = feature_maps[0]['out_conv7_2'][0]

  rfem11 = torch.add(rfem(conv4_2), conv4_3)
  rfem12 = torch.add(rfem(rfem11), conv5_3)
  rfem13 = torch.add(rfem(rfem12), conv7_2)
  rfem14 = torch.add(rfem(rfem13), conv6_2)

  ftb_4 = ftb(rfem14)
  ftb_3 = ftb(rfem13)
  ftb_2 = ftb(rfem12)
  ftb_1 = ftb(rfem11)

  rfem21 = torch.add(rfem(ftb_1), conv4_3)
  rfem22 = torch.add(rfem(ftb_2), conv5_3)
  rfem23 = torch.add(rfem(ftb_3), conv7_2)
  rfem24 = torch.add(rfem(ftb_4), conv6_2)

  det_1 = torch.nn.Softmax(dim=1)(torch.cat((rfem21, rfem11)))
  det_2 = torch.nn.Softmax(dim=1)(torch.cat((rfem22, rfem12)))
  det_3 = torch.nn.Softmax(dim=1)(torch.cat((rfem23, rfem13)))
  det_4 = torch.nn.Softmax(dim=1)(torch.cat((rfem24, rfem14)))

  D = torch.cat((det_1, det_2, det_3, det_4), dim=1)
  return D

In [51]:
D = training(x)

tensor([[[[9.8675e-06, 7.6592e-05, 7.9785e-05,  ..., 3.6967e-05,
           5.5655e-03, 1.5274e-03],
          [1.0576e-03, 3.0208e-04, 3.3255e-05,  ..., 1.2904e-04,
           1.6082e-04, 3.1886e-05],
          [1.0553e-06, 3.7846e-04, 2.8294e-04,  ..., 4.7358e-04,
           3.7130e-03, 2.3129e-03],
          ...,
          [1.6168e-04, 2.6659e-03, 1.1396e-02,  ..., 1.8870e-04,
           4.7930e-04, 5.8423e-04],
          [1.4415e-04, 6.4390e-05, 9.3836e-05,  ..., 2.2485e-04,
           5.5638e-04, 3.4276e-04],
          [7.1121e-04, 2.5477e-04, 1.4583e-04,  ..., 7.1426e-05,
           2.8812e-03, 6.1258e-04]],

         [[2.8202e-05, 1.1392e-04, 8.3681e-05,  ..., 3.8878e-05,
           1.5332e-04, 4.8213e-05],
          [1.9292e-04, 1.6619e-04, 4.7490e-05,  ..., 1.4507e-04,
           1.4916e-04, 2.4452e-05],
          [9.9802e-01, 1.2511e-04, 1.8806e-04,  ..., 2.0806e-04,
           1.0714e-01, 2.3498e-04],
          ...,
          [2.7599e-04, 6.9536e-05, 3.7427e-04,  ..., 3.3682